In [1]:
import os
import numpy as np
import pandas as pd
import flair
import datetime
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm

In [2]:
df = pd.read_csv('Reddit_data_Binance.csv')
df.head()

,post_id,title,selftext,url,author,score,publish_date,num_of_comments,permalink,flair
0,rt08h3,Transferring ADA back to Binance from Binance ...,"Hello guys,\n\nRecently, I transferred my ADA ...",https://www.reddit.com/r/binance/comments/rt08...,samilniftaliyev,1,2022-01-01 00:04:39,0,/r/binance/comments/rt08h3/transferring_ada_ba...,Binance.com
1,rt0akw,IPad Binance App,NaN,https://www.reddit.com/r/binance/comments/rt0a...,sanchit123k,1,2022-01-01 00:07:44,0,/r/binance/comments/rt0akw/ipad_binance_app/,General
2,rt0b8v,Have you ever sent coins from Metamask into th...,NaN,https://www.reddit.com/r/binance/comments/rt0b...,VittorSilva,1,2022-01-01 00:08:44,0,/r/binance/comments/rt0b8v/have_you_ever_sent_...,General
3,rt0ppu,"Binance has closed its China market, will this...",NaN,https://i.redd.it/33t8y8ysdx881.jpg,Christina21000,1,2022-01-01 00:28:52,0,/r/binance/comments/rt0ppu/binance_has_closed_...,General
4,rt1e7y,Ontario securities regulator slams Binance aft...,NaN,https://www.theblockcrypto.com/post/128831/ont...,the_dreamer2020,1,2022-01-01 01:01:44,0,/r/binance/comments/rt1e7y/ontario_securities_...,General


In [3]:
df = df[['title','selftext','publish_date']]
df['date'] = df['publish_date']
df.drop('publish_date',axis=1,inplace=True)
df = df.fillna('')
df['text'] = df['title'] + ' ' + df['selftext']
df.set_index('date', inplace=True)

In [4]:
df.drop(['title','selftext'], axis=1, inplace=True)

In [5]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
fmt = '%Y-%m-%d %H:00:00'
vader = SentimentIntensityAnalyzer()

2022-05-10 13:10:37,967 loading file C:\Users\mpmur\.flair\models\sentiment-en-mix-distillbert_4.pt


In [6]:
def sentiment_val_flair(flair_tag,flair_score):
    neg = 'NEGATIVE' in flair_tag
    val = float(flair_score)
    if neg:
        return -val
    return val

In [7]:
def get_sentiment_report(df):
    tb_sentiment_polarity_ = []
    tb_sentiment_subjectivity_ = []
    flair_sentiment_ = []
    pos_ = []
    neg_ = []
    neu_ = []
    com_ = []
    for i,j in enumerate(tqdm(df['text'])):
        v = vader.polarity_scores(j)
        pos_.append(v['pos'])
        neg_.append(v['neg'])
        neu_.append(v['neu'])
        com_.append(v['compound'])
        flair_s = flair.data.Sentence(j)
        flair_sentiment.predict(flair_s)
        flair_tag = flair_s.tag
        flair_score = flair_s.score
        flair_val = sentiment_val_flair(flair_tag,flair_score)
        flair_sentiment_.append(flair_val)
        tb_sentiment_polarity_.append(TextBlob(j).sentiment[0])
        tb_sentiment_subjectivity_.append(TextBlob(j).sentiment[1])
        
    
    df['Reddit_Flair'] = flair_sentiment_
    df['Reddit_TB_Polarity'] = tb_sentiment_polarity_
    df['Reddit_TB_Subjectivity'] = tb_sentiment_subjectivity_
    df['Reddit_Vader_Pos'] = pos_
    df['Reddit_Vader_Neg'] = neg_
    df['Reddit_Vader_Neu'] = neu_
    df['Reddit_Vader_Compound'] = com_
    df.to_csv('Reddit_Sentiment_BNB.csv')

In [8]:
get_sentiment_report(df)

100%|██████████████████████████████████████████████████████████████████████████████| 3164/3164 [06:06<00:00,  8.64it/s]
